In [168]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression


In [169]:
df = pd.read_csv('car_insurance.csv')

In [170]:
df.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.000000,after 2015,0.000000,1.000000,10238,12000.000000,sedan,0,0,0,0.000000
1,750365,0,1,0-9y,none,poverty,0.357757,0.000000,before 2015,0.000000,0.000000,10238,16000.000000,sedan,0,0,0,1.000000
2,199901,0,0,0-9y,high school,working class,0.493146,1.000000,before 2015,0.000000,0.000000,10238,11000.000000,sedan,0,0,0,0.000000
3,478866,0,1,0-9y,university,working class,0.206013,1.000000,before 2015,0.000000,1.000000,32765,11000.000000,sedan,0,0,0,0.000000
4,731664,1,1,10-19y,none,working class,0.388366,1.000000,before 2015,0.000000,0.000000,32765,12000.000000,sedan,2,0,1,1.000000


### EDA
- Check for missing values
- Fill missing values with the mean
- Check whether standardization is needed
- Check individual features' impact on the outcome and model accuracy

In [171]:
df.describe()

,id,age,gender,credit_score,vehicle_ownership,married,children,postal_code,annual_mileage,speeding_violations,duis,past_accidents,outcome
count,10000.000000,10000.000000,10000.000000,9018.000000,10000.000000,10000.000000,10000.000000,10000.000000,9043.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,500521.906800,1.489500,0.499000,0.515813,0.697000,0.498200,0.688800,19864.548400,11697.003207,1.482900,0.239200,1.056300,0.313300
std,290030.768758,1.025278,0.500024,0.137688,0.459578,0.500022,0.463008,18915.613855,2818.434528,2.241966,0.554990,1.652454,0.463858
min,101.000000,0.000000,0.000000,0.053358,0.000000,0.000000,0.000000,10238.000000,2000.000000,0.000000,0.000000,0.000000,0.000000
25%,249638.500000,1.000000,0.000000,0.417191,0.000000,0.000000,0.000000,10238.000000,10000.000000,0.000000,0.000000,0.000000,0.000000
50%,501777.000000,1.000000,0.000000,0.525033,1.000000,0.000000,1.000000,10238.000000,12000.000000,0.000000,0.000000,0.000000,0.000000
75%,753974.500000,2.000000,1.000000,0.618312,1.000000,1.000000,1.000000,32765.000000,14000.000000,2.000000,0.000000,2.000000,1.000000
max,999976.000000,3.000000,1.000000,0.960819,1.000000,1.000000,1.000000,92101.000000,22000.000000,22.000000,6.000000,15.000000,1.000000


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [173]:
df['credit_score'].fillna(df['credit_score'].mean(), inplace=True)
df['annual_mileage'].fillna(df['annual_mileage'].mean(), inplace=True)

In [174]:
X = df.drop(columns=['id','outcome'])
Y = df['outcome']

In [175]:
# Standardization
scaler = StandardScaler()
def scale_numeric(col):
    if col.dtype in ['float64', 'int64']:  # Check if the column is numeric
        return scaler.fit_transform(col.values.reshape(-1, 1)).flatten()
    else:
        return col
    
X_standardized = X.apply(scale_numeric)

In [176]:
X_standardized.head(5)

,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents
0,1.473333,-0.998002,0-9y,high school,upper class,0.865914,0.659333,after 2015,-0.996406,0.672161,-0.508946,0.113057,sedan,-0.661462,-0.431020,-0.639263
1,-1.452849,1.002002,0-9y,none,poverty,-1.208879,-1.516684,before 2015,-0.996406,-1.487739,-0.508946,1.605576,sedan,-0.661462,-0.431020,-0.639263
2,-1.452849,-0.998002,0-9y,high school,working class,-0.173367,0.659333,before 2015,-0.996406,-1.487739,-0.508946,-0.260073,sedan,-0.661462,-0.431020,-0.639263
3,-1.452849,1.002002,0-9y,university,working class,-2.369485,0.659333,before 2015,-0.996406,0.672161,0.682034,-0.260073,sedan,-0.661462,-0.431020,-0.639263
4,-0.477455,1.002002,10-19y,none,working class,-0.974770,0.659333,before 2015,-0.996406,-1.487739,0.682034,0.113057,sedan,0.230657,-0.431020,-0.034072


In [177]:
# Empty list to store model results
models = []

# Loop through features
for col in X_standardized.columns:
    # Create a model
    model = logit(f"outcome ~ {col}", data=df).fit()
    # Add each model to the models list
    models.append(model)

Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

In [178]:
# Empty list to store model accuracies
accuracies = []

# Loop through models
for feature in range(0, len(models)):
    # Compute the confusion matrix
    conf_matrix = models[feature].pred_table()
    # True negatives
    tn = conf_matrix[0,0]
    # True positives
    tp = conf_matrix[1,1]
    # False negatives
    fn = conf_matrix[1,0]
    # False positives
    fp = conf_matrix[0,1]
    # Compute accuracy
    acc = (tn + tp) / (tn + fn + fp + tp)
    accuracies.append(acc)

In [179]:
from sklearn.metrics import precision_recall_fscore_support

# Lists to store results
accuracies = []
p_values = []
auc_roc_scores = []
true_negatives = []
false_positives = []
false_negatives = []
true_positives = []
precision_0 = []
recall_0 = []
precision_1 = []
recall_1 = []
f1_score_0 = []
f1_score_1 = []

# Loop through features
for col in X_standardized.columns:
    # Create and fit the model
    model = logit(f"outcome ~ {col}", data=df).fit()
    
    # Predictions
    y_pred_proba = model.predict(df[col])
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    
    # Accuracy
    accuracy = (tn + tp) / (tn + fp + fn + tp)
    accuracies.append(accuracy)
    
    # P-value
    p_values.append(model.pvalues[1])
    
    # AUC-ROC
    auc_roc = roc_auc_score(Y, y_pred_proba)
    auc_roc_scores.append(auc_roc)
    
    # Confusion Matrix values
    true_negatives.append(tn)
    false_positives.append(fp)
    false_negatives.append(fn)
    true_positives.append(tp)
    
    # Precision, Recall, F1-Score
    prec, rec, f1, _ = precision_recall_fscore_support(Y, y_pred, average=None)
    precision_0.append(prec[0])
    recall_0.append(rec[0])
    precision_1.append(prec[1])
    recall_1.append(rec[1])
    f1_score_0.append(f1[0])
    f1_score_1.append(f1[1])

# Create the dataframe
results_df = pd.DataFrame({
    'feature': X_standardized.columns,
    'accuracy': accuracies,
    'p_value': p_values,
    'auc_roc': auc_roc_scores,
    'true_negative': true_negatives,
    'false_positive': false_positives,
    'false_negative': false_negatives,
    'true_positive': true_positives,
    'precision_0': precision_0,
    'recall_0': recall_0,
    'precision_1': precision_1,
    'recall_1': recall_1,
    'f1_score_0': f1_score_0,
    'f1_score_1': f1_score_1
})

# Sort the dataframe by AUC-ROC in descending order
results_df = results_df.sort_values('auc_roc', ascending=False).reset_index(drop=True)



Optimization terminated successfully.
         Current function value: 0.511794
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8


/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5


/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

Optimization terminated successfully.
         Current function value: 0.595431
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.617345
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.605716
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621700
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.558922
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.598699
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.549220
         Iterations 7


/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tonychu/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

In [180]:
results_df 

,feature,accuracy,p_value,auc_roc,true_negative,false_positive,false_negative,true_positive,precision_0,recall_0,precision_1,recall_1,f1_score_0,f1_score_1
0,driving_experience,0.777100,0.000000,0.808000,5554,1313,916,2217,0.858423,0.808796,0.628045,0.707628,0.832871,0.665466
1,age,0.774700,0.000000,0.769057,6299,568,1685,1448,0.788953,0.917286,0.718254,0.462177,0.848293,0.562439
2,income,0.742500,0.000000,0.744600,6239,628,1947,1186,0.762155,0.908548,0.653804,0.378551,0.828938,0.479483
3,speeding_violations,0.686700,0.000000,0.718346,6867,0,3133,0,0.686700,1.000000,0.000000,0.000000,0.814253,0.000000
4,past_accidents,0.686700,0.000000,0.712606,6867,0,3133,0,0.686700,1.000000,0.000000,0.000000,0.814253,0.000000
5,credit_score,0.705400,0.000000,0.693267,6321,546,2400,733,0.724802,0.920489,0.573104,0.233961,0.811008,0.332276
6,vehicle_ownership,0.735100,0.000000,0.687712,5594,1273,1376,1757,0.802582,0.814621,0.579868,0.560804,0.808557,0.570177
7,vehicle_year,0.686700,0.000000,0.645772,6867,0,3133,0,0.686700,1.000000,0.000000,0.000000,0.814253,0.000000
8,married,0.686700,0.000000,0.641269,6867,0,3133,0,0.686700,1.000000,0.000000,0.000000,0.814253,0.000000
9,children,0.686700,0.000000,0.616204,6867,0,3133,0,0.686700,1.000000,0.000000,0.000000,0.814253,0.000000


### Model Building
- Build a model with the best features
- Evaluate the model's performance
- Tune the model's performance
- Evaluate the model's performance

In [181]:
# List of features you want to include
features = X_standardized.columns

# Join the features into a single string separated by +
features_str = " + ".join(features)

# Create the formula
formula = f"outcome ~ {features_str}"

model = logit(formula, data=df).fit()

Optimization terminated successfully.
         Current function value: 0.357676
         Iterations 8


In [182]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                outcome   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9978
Method:                           MLE   Df Model:                           21
Date:                Sun, 25 Aug 2024   Pseudo R-squ.:                  0.4247
Time:                        17:33:53   Log-Likelihood:                -3576.8
converged:                       True   LL-Null:                       -6217.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -1.1689      0.287     -4.078      0.000      -1.731      -0.607
driving_experience[T.10-19y]    -1.8597      0.084    -22.075      0.000      -2.025      -1.695
driving_experience[T.20-29y]    -3.4535      0.163    -21.236      0.000      -3.772      -3.135
driving_experience[T.30y+]      -4.0486      0.301    -13.453      0.000      -4.638      -3.459
education[T.none]                0.0409      0.083      0.494      0.621      -0.121       0.203
education[T.university]          0.0176      0.075      0.235      0.814      -0.129       0.165
income[T.poverty]                0.0761      0.115      0.660      0.509      -0.150       0.302
income[T.upper class]           -0.0226      0.099     -0.229      0.819      -0.216       0.171
income[T.working class]          0.0754      0.095      0.797      0.425      -0.110       0.261
vehicle_year[T.before 2015]      1.7297      0.083     20.944      0.000       1.568       1.892
vehicle_type[T.sports car]       0.0215      0.138      0.156      0.876      -0.248       0.291
age                             -0.0322      0.047     -0.689      0.491      -0.124       0.059
gender                           0.9594      0.064     14.892      0.000       0.833       1.086
credit_score                     0.1123      0.324      0.347      0.729      -0.522       0.747
vehicle_ownership               -1.6922      0.068    -25.038      0.000      -1.825      -1.560
married                         -0.4131      0.070     -5.904      0.000      -0.550      -0.276
children                        -0.1576      0.070     -2.256      0.024      -0.295      -0.021
postal_code                   2.122e-05   1.61e-06     13.190      0.000    1.81e-05    2.44e-05
annual_mileage                 6.71e-05   1.37e-05      4.894      0.000    4.02e-05     9.4e-05
speeding_violations              0.0537      0.024      2.201      0.028       0.006       0.102
duis                             0.0533      0.075      0.707      0.480      -0.094       0.201
past_accidents                  -0.1523      0.036     -4.278      0.000      -0.222      -0.083
================================================================================================
"""

In [183]:
X = df.drop(columns=['id','outcome'])
Y = df['outcome']

# Standardization
scaler = StandardScaler()
def scale_numeric(col):
    if col.dtype in ['float64', 'int64']:  # Check if the column is numeric
        return scaler.fit_transform(col.values.reshape(-1, 1)).flatten()
    else:
        return col
    
X_standardized = X.apply(scale_numeric)

In [184]:
new_df = df.drop(columns=['education', 'income', 'vehicle_type', 'age', 'credit_score','duis'])
new_X = new_df.drop(columns = ['id','outcome'])
new_Y = new_df['outcome']

# Standardization
scaler = StandardScaler()
def scale_numeric(col):
    if col.dtype in ['float64', 'int64']:  # Check if the column is numeric
        return scaler.fit_transform(col.values.reshape(-1, 1)).flatten()
    else:
        return col
    
new_X_standardized = new_X.apply(scale_numeric)


new_X_standardized = pd.get_dummies(new_X_standardized, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(new_X_standardized, new_Y, test_size=0.3, random_state=42)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)




In [185]:
# Ensure all boolean columns are converted to integers (0 and 1)
X_train_clean = X_train_const.copy()
boolean_columns = X_train_clean.select_dtypes(include=['bool']).columns
X_train_clean[boolean_columns] = X_train_clean[boolean_columns].astype(int)

# Re-add the constant column, if necessary
if 'const' not in X_train_clean.columns:
    X_train_clean = sm.add_constant(X_train_clean)

In [186]:
# Ensure all boolean columns are converted to integers (0 and 1)

X_test_clean = X_test_const.copy()
boolean_columns = X_test_clean.select_dtypes(include=['bool']).columns
X_test_clean[boolean_columns] = X_test_clean[boolean_columns].astype(int)

# Re-add the constant column, if necessary
if 'const' not in X_test_clean.columns:
    X_test_clean = sm.add_constant(X_test_clean)

In [187]:
# Fit the logistic regression model on the training data
# Initialize the LogisticRegression model
model = LogisticRegression()

# Fit the model on the training data
results = model.fit(X_train_clean, y_train)

# Predict the labels for the test set
y_pred = model.predict(X_test_clean)

In [188]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

print("Classification Report:\n", classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# AUC-ROC score
y_pred_prob = model.predict_proba(X_test_clean)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC-ROC: {roc_auc:.4f}')

Accuracy: 0.8453
Classification Report:
               precision    recall  f1-score   support

         0.0       0.88      0.90      0.89      2063
         1.0       0.77      0.72      0.74       937

    accuracy                           0.85      3000
   macro avg       0.82      0.81      0.82      3000
weighted avg       0.84      0.85      0.84      3000

Confusion Matrix:
[[1860  203]
 [ 261  676]]
AUC-ROC: 0.9049


In [189]:
# Get the coefficients and intercept from the sklearn model
coef = model.coef_.flatten()
intercept = model.intercept_[0]

# Combine intercept and coefficients
all_coef = np.concatenate(([intercept], coef))

# Get feature names (including a name for the intercept)
feature_names = ['intercept'] + list(model.feature_names_in_)

# Calculate standard errors
pred = model.predict(X_test_clean)
residuals = y_test - pred
mse = np.mean(residuals**2)
var_coef = mse * np.linalg.inv(np.dot(X_test_clean.T, X_test_clean)).diagonal()
std_errors = np.sqrt(var_coef)

# Ensure std_errors has the same length as all_coef
if len(std_errors) < len(all_coef):
    std_errors = np.concatenate(([np.nan], std_errors))

# Calculate z-scores and p-values
z_scores = all_coef / std_errors
p_values = [2 * (1 - stats.norm.cdf(np.abs(z))) for z in z_scores]

# Create summary DataFrame
summary_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': all_coef,
    'Std Error': std_errors,
    'z-value': z_scores,
    'p-value': p_values
})

def format_float(x):
    if isinstance(x, (float, np.float64)):
        return f"{x:.6f}"
    return x

# Apply formatting to all columns except 'Feature'
for col in summary_df.columns:
    if col != 'Feature':
        summary_df[col] = summary_df[col].apply(format_float)

# Display the summary table
pd.set_option('display.float_format', lambda x: '%.6f' % x)


In [190]:
summary_df

,Feature,Coefficient,Std Error,z-value,p-value
0,intercept,-1.074263,nan,nan,nan
1,const,-0.001625,0.019604,-0.082904,0.933928
2,gender,0.491157,0.007749,63.382176,0.000000
3,vehicle_ownership,-0.789395,0.007488,-105.421174,0.000000
4,married,-0.220251,0.008307,-26.512893,0.000000
5,children,-0.085727,0.008236,-10.408779,0.000000
6,postal_code,0.417328,0.007200,57.959658,0.000000
7,annual_mileage,0.172608,0.008792,19.632080,0.000000
8,speeding_violations,0.084093,0.010117,8.312333,0.000000
9,past_accidents,-0.316833,0.009784,-32.384295,0.000000


## ANALYSIS

### 1. Model Performance:
- Accuracy: 0.8453 (84.53%), up from 79.57%
- AUC-ROC: 0.9049 (90.49%), up from 0.8201
These metrics indicate excellent predictive power. The AUC-ROC of 0.9049 suggests the model has very strong discriminative ability between classes.
### 2. Classification Report:
- For non-claims (0.0):
Precision: 0.88, Recall: 0.90, F1-score: 0.89
- For claims (1.0):
Precision: 0.77, Recall: 0.72, F1-score: 0.74

The model now performs well on both classes, with a more balanced performance between claims and non-claims.

### 3. Confusion Matrix:
[[1860 203]
[ 261 676]]
- True Negatives: 1860 (up from 1828)
- False Positives: 203 (down from 235)
- False Negatives: 261 (down from 378)
- True Positives: 676 (up from 559)

The model has improved in all aspects, particularly in reducing false negatives.

### 4. Feature Importance:
All features except 'const' are statistically significant (p-value < 0.05). Key findings:
- Gender is now significant and positively correlated with claims.
- Vehicle ownership remains negatively correlated with claims.
- Driving experience shows a strong negative correlation with claims, increasing with experience.
- Older vehicles (before 2015) are strongly associated with more claims.
- Speeding violations now show a positive correlation with claims, which is more intuitive than the previous result.
- Past accidents still show a negative correlation, which remains counterintuitive and might need further investigation.

### 5. Prediction Bias:

The model now shows more balanced performance between classes, with the gap in recall between non-claims (0.90) and claims (0.72) reduced compared to the previous model.

Interpretation:

1. Overall Improvement: The model's performance has significantly improved across all metrics.
2. Better Balance: The model now handles both classes more effectively, reducing the prediction bias observed earlier.
3. Feature Relationships: Most feature relationships are now more intuitive, except for past accidents.
4. Driving Experience: There's a clear trend showing that more experienced drivers are less likely to make claims.
5. Vehicle Age: Older vehicles are strongly associated with higher claim likelihood.

Recommendations:

1. Model Validation: Perform cross-validation to ensure these results are consistent across different data subsets.
2. Further Investigation: Look into the negative correlation between past accidents and claims, as this remains counterintuitive.
3. Feature Engineering: Consider creating interaction terms, especially between driving experience and other risk factors.
4. Threshold Tuning: While performance is more balanced, you might still want to fine-tune the classification threshold based on the relative costs of false positives vs. false negatives.
5. Model Deployment: Given the significant improvement, this model could be considered for deployment after thorough validation and any necessary regulatory approvals.
6. Monitoring: Implement a system to monitor the model's performance over time to ensure it maintains its predictive power in real-world applications.
This improved model provides a strong foundation for predicting insurance claims, with good performance across both classes and mostly intuitive feature relationships.